Libraries - see requirements.txt

In [ ]:
# !pip install investpy # https://investpy.readthedocs.io/
!pip install yfinance # https://pypi.org/project/yfinance/
!pip install hurst # https://pypi.org/project/hurst/
!pip install pystan==2.19.1.1
!pip install prophet #https://facebook.github.io/prophet/docs/installation.html

Import libraries

In [2]:
# import investpy
import yfinance as yf
import pandas_datareader.data as web
from hurst import compute_Hc, random_walk
from prophet import Prophet
import pandas as pd
import numpy as np
import datetime

Import symbols.

In [47]:
symbols_dict_url = 'https://raw.githubusercontent.com/jefferygao1984/METIS_IDS_PROJECT/master/symbols_dict_02.csv'
symbols_dict = pd.read_csv(symbols_dict_url)
symbols_dict['LABEL'] = symbols_dict['TICKER_TYPE'] + ' : ' + symbols_dict['TICKER']
symbols_dict = symbols_dict.tail(5)
symbols_dict

,TICKER,SOURCE,TICKER_TYPE,DESCRIPTION,QUAD1_GOLDILOCKS,QUAD2_REFLATION,QUAD3_STAGFLATION,QUAD4_DEFLATION,LABEL
56,XMR-USD,YAHOO,CRYPTO,Monero USD,1,1,0,0,CRYPTO : XMR-USD
57,^VIX,YAHOO,INDICES,CBOE Volatility Index,0,0,1,1,INDICES : ^VIX
58,^VVIX,YAHOO,INDICES,CBOE VIX VOLATILITY INDEX,0,0,1,1,INDICES : ^VVIX
59,^VXN,YAHOO,INDICES,CBOE NASDAQ 100 Voltility,0,0,1,1,INDICES : ^VXN
60,^OVX,YAHOO,INDICES,CBOE Crude Oil Volatility Index,0,0,1,1,INDICES : ^OVX


In [132]:
t0 = datetime.datetime.now()

intervals = ['15m','60m','1d']    # '15m' = 60 days , '60m' = 730 days , '1d' = unlimited
now = datetime.datetime.now()
end = str(now.year)+"-"+str(now.month)+"-"+str(now.day)

data_combined_15m = pd.DataFrame()
data_combined_60m = pd.DataFrame()
data_combined_daily = pd.DataFrame()

for interval in intervals:
    for ticker in symbols_dict['TICKER']:

        current_time = datetime.datetime.now()
        current_time = str(current_time)

        if interval == '15m':
            start = now - datetime.timedelta(days=60-1)    # max = 60 - 1
            start = str(start.year)+"-"+str(start.month)+"-"+str(start.day)
        elif interval == '60m':
            start = now - datetime.timedelta(days=730-1)    # max = 730 - 1
            start = str(start.year)+"-"+str(start.month)+"-"+str(start.day)
        else:
            start = '1900-01-01'

        temp_df = yf.download(ticker, start=start, end=end, interval=interval, prepost = False, threads = True, proxy = None)
        temp_df['OHLC'] = 0.25 * (temp_df['Open'] + temp_df['High'] + temp_df['Low'] + temp_df['Close'])
        temp_df['TICKER'] = ticker
        temp_df['LABEL'] = symbols_dict.loc[symbols_dict['TICKER'] == ticker, 'LABEL'].iloc[0]

        if interval == '15m':
            frames = [data_combined_15m, temp_df]
            data_combined_15m = pd.concat(frames)
        elif interval == '60m':
            frames = [data_combined_60m, temp_df]
            data_combined_60m = pd.concat(frames)
        else:
            frames = [data_combined_daily, temp_df]
            data_combined_daily = pd.concat(frames)

data_combined_15m.columns = data_combined_15m.columns.str.replace(' ', '_')
data_combined_60m.columns = data_combined_60m.columns.str.replace(' ', '_')
data_combined_daily.columns = data_combined_daily.columns.str.replace(' ', '_')

data_combined_15m.reset_index(inplace=True)
data_combined_60m.reset_index(inplace=True)
data_combined_daily.reset_index(inplace=True)

data_combined_15m['idx'] = data_combined_15m['Datetime']
data_combined_60m['idx'] = data_combined_60m['Datetime']
data_combined_daily['idx'] = data_combined_daily['Date']
data_combined_15m = data_combined_15m.set_index('idx')
data_combined_60m = data_combined_60m.set_index('idx')
data_combined_daily = data_combined_daily.set_index('idx')

'''
data_combined_15m['Volume'] = data_combined_15m['Volume'].fillna(1)
data_combined_60m['Volume'] = data_combined_60m['Volume'].fillna(1)
data_combined_daily['Volume'] = data_combined_daily['Volume'].fillna(1)
'''

'''
data_combined_15m['Volume'] = data_combined_15m['Volume'].replace(0, 1)
data_combined_60m['Volume'] = data_combined_60m['Volume'].replace(0, 1)
data_combined_daily['Volume'] = data_combined_daily['Volume'].replace(0, 1)

data_combined_15m['OHLC'] = 0.25 * (data_combined_15m['Open'] + data_combined_15m['High'] + data_combined_15m['Low'] + data_combined_15m['Close'])
data_combined_60m['OHLC'] = 0.25 * (data_combined_60m['Open'] + data_combined_60m['High'] + data_combined_60m['Low'] + data_combined_60m['Close'])
data_combined_daily['OHLC'] = 0.25 * (data_combined_daily['Open'] + data_combined_daily['High'] + data_combined_daily['Low'] + data_combined_daily['Close'])
'''

t0 = datetime.datetime.now() - t0
print('Section exec time: ', t0,'s')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Refactor from long to wide.

Also note the [groupby + aggregate + unstack] method.

https://stackoverflow.com/questions/26255671/pandas-column-values-to-columns

temp = data_combined_15m[['Adj_Close','TICKER']]

data_combined_15m_refactored = temp.reset_index().groupby(['idx', 'TICKER'])['Adj Close'].aggregate('first').unstack()

In [140]:
# https://stackoverflow.com/questions/26255671/pandas-column-values-to-columns

t0 = datetime.datetime.now()

'''
temp = data_combined_15m[['Open','TICKER']]
data_combined_15m_open_refactored = temp.pivot_table(values='Open', index=temp.index, columns='TICKER', aggfunc='first')
data_combined_15m_open_refactored = data_combined_15m_open_refactored.fillna(method='ffill')
data_combined_15m_open_refactored = data_combined_15m_open_refactored.dropna()

temp = data_combined_60m[['Open','TICKER']]
data_combined_60m_open_refactored = temp.pivot_table(values='Open', index=temp.index, columns='TICKER', aggfunc='first')
data_combined_60m_open_refactored = data_combined_60m_open_refactored.fillna(method='ffill')
data_combined_60m_open_refactored = data_combined_60m_open_refactored.dropna()

temp = data_combined_daily[['Open','TICKER']]
data_combined_daily_open_refactored = temp.pivot_table(values='Open', index=temp.index, columns='TICKER', aggfunc='first')
data_combined_daily_open_refactored = data_combined_daily_open_refactored.fillna(method='ffill')
data_combined_daily_open_refactored = data_combined_daily_open_refactored.dropna()

temp = data_combined_15m[['High','TICKER']]
data_combined_15m_high_refactored = temp.pivot_table(values='High', index=temp.index, columns='TICKER', aggfunc='first')
data_combined_15m_high_refactored = data_combined_15m_high_refactored.fillna(method='ffill')
data_combined_15m_high_refactored = data_combined_15m_high_refactored.dropna()

temp = data_combined_60m[['High','TICKER']]
data_combined_60m_high_refactored = temp.pivot_table(values='High', index=temp.index, columns='TICKER', aggfunc='first')
data_combined_60m_high_refactored = data_combined_60m_high_refactored.fillna(method='ffill')
data_combined_60m_high_refactored = data_combined_60m_high_refactored.dropna()

temp = data_combined_daily[['High','TICKER']]
data_combined_daily_high_refactored = temp.pivot_table(values='High', index=temp.index, columns='TICKER', aggfunc='first')
data_combined_daily_high_refactored = data_combined_daily_high_refactored.fillna(method='ffill')
data_combined_daily_high_refactored = data_combined_daily_high_refactored.dropna()

temp = data_combined_15m[['Low','TICKER']]
data_combined_15m_low_refactored = temp.pivot_table(values='Low', index=temp.index, columns='TICKER', aggfunc='first')
data_combined_15m_low_refactored = data_combined_15m_low_refactored.fillna(method='ffill')
data_combined_15m_low_refactored = data_combined_15m_low_refactored.dropna()

temp = data_combined_60m[['Low','TICKER']]
data_combined_60m_low_refactored = temp.pivot_table(values='Low', index=temp.index, columns='TICKER', aggfunc='first')
data_combined_60m_low_refactored = data_combined_60m_low_refactored.fillna(method='ffill')
data_combined_60m_low_refactored = data_combined_60m_low_refactored.dropna()

temp = data_combined_daily[['Low','TICKER']]
data_combined_daily_low_refactored = temp.pivot_table(values='Low', index=temp.index, columns='TICKER', aggfunc='first')
data_combined_daily_low_refactored = data_combined_daily_low_refactored.fillna(method='ffill')
data_combined_daily_low_refactored = data_combined_daily_low_refactored.dropna()

temp = data_combined_15m[['Close','TICKER']]
data_combined_15m_close_refactored = temp.pivot_table(values='Close', index=temp.index, columns='TICKER', aggfunc='first')
data_combined_15m_close_refactored = data_combined_15m_close_refactored.fillna(method='ffill')
data_combined_15m_close_refactored = data_combined_15m_close_refactored.dropna()

temp = data_combined_60m[['Close','TICKER']]
data_combined_60m_close_refactored = temp.pivot_table(values='Close', index=temp.index, columns='TICKER', aggfunc='first')
data_combined_60m_close_refactored = data_combined_60m_close_refactored.fillna(method='ffill')
data_combined_60m_close_refactored = data_combined_60m_close_refactored.dropna()

temp = data_combined_daily[['Close','TICKER']]
data_combined_daily_close_refactored = temp.pivot_table(values='Close', index=temp.index, columns='TICKER', aggfunc='first')
data_combined_daily_close_refactored = data_combined_daily_close_refactored.fillna(method='ffill')
data_combined_daily_close_refactored = data_combined_daily_close_refactored.dropna()

temp = data_combined_15m[['Adj_Close','TICKER']]
data_combined_15m_adjclose_refactored = temp.pivot_table(values='Adj_Close', index=temp.index, columns='TICKER', aggfunc='first')
data_combined_15m_adjclose_refactored = data_combined_15m_adjclose_refactored.fillna(method='ffill')
data_combined_15m_adjclose_refactored = data_combined_15m_adjclose_refactored.dropna()

temp = data_combined_60m[['Adj_Close','TICKER']]
data_combined_60m_adjclose_refactored = temp.pivot_table(values='Adj_Close', index=temp.index, columns='TICKER', aggfunc='first')
data_combined_60m_adjclose_refactored = data_combined_60m_adjclose_refactored.fillna(method='ffill')
data_combined_60m_adjclose_refactored = data_combined_60m_adjclose_refactored.dropna()

temp = data_combined_daily[['Adj_Close','TICKER']]
data_combined_daily_adjclose_refactored = temp.pivot_table(values='Adj_Close', index=temp.index, columns='TICKER', aggfunc='first')
data_combined_daily_adjclose_refactored = data_combined_daily_adjclose_refactored.fillna(method='ffill')
data_combined_daily_adjclose_refactored = data_combined_daily_adjclose_refactored.dropna()
'''

temp = data_combined_15m[['OHLC','LABEL']]
data_combined_15m_OHLC_refactored = temp.pivot_table(values='OHLC', index=temp.index, columns='LABEL', aggfunc='first')

temp = data_combined_60m[['OHLC','LABEL']]
data_combined_60m_OHLC_refactored = temp.pivot_table(values='OHLC', index=temp.index, columns='LABEL', aggfunc='first')

temp = data_combined_daily[['OHLC','LABEL']]
data_combined_daily_OHLC_refactored = temp.pivot_table(values='OHLC', index=temp.index, columns='LABEL', aggfunc='first')

temp = data_combined_15m[['Volume','LABEL']]
data_combined_15m_volume_refactored = temp.pivot_table(values='Volume', index=temp.index, columns='LABEL', aggfunc='first')

temp = data_combined_60m[['Volume','LABEL']]
data_combined_60m_volume_refactored = temp.pivot_table(values='Volume', index=temp.index, columns='LABEL', aggfunc='first')

temp = data_combined_daily[['Volume','LABEL']]
data_combined_daily_volume_refactored = temp.pivot_table(values='Volume', index=temp.index, columns='LABEL', aggfunc='first')

t0 = datetime.datetime.now() - t0
print('Section exec time: ', t0,'s')

Section exec time:  0:00:00.538243 s


In [124]:
data_combined_15m.TICKER.unique()

array(['XMR-USD', '^VIX', '^VVIX', '^VXN', '^OVX'], dtype=object)

In [ ]:
data_combined_15m[data_combined_15m['TICKER'] == '^VIX']

In [136]:
# fill and replace

data_combined_15m_OHLC_refactored = data_combined_15m_OHLC_refactored.fillna(method='ffill')
data_combined_15m_OHLC_refactored = data_combined_15m_OHLC_refactored.fillna(method='bfill')
data_combined_60m_OHLC_refactored = data_combined_60m_OHLC_refactored.fillna(method='ffill')
data_combined_60m_OHLC_refactored = data_combined_60m_OHLC_refactored.fillna(method='bfill')
data_combined_daily_OHLC_refactored = data_combined_daily_OHLC_refactored.fillna(method='ffill')
data_combined_daily_OHLC_refactored = data_combined_daily_OHLC_refactored.fillna(method='bfill')

data_combined_15m_volume_refactored = data_combined_15m_volume_refactored.fillna(method='ffill')
data_combined_15m_volume_refactored = data_combined_15m_volume_refactored.fillna(method='bfill')
data_combined_15m_volume_refactored = data_combined_15m_volume_refactored.replace(0, 1)
data_combined_15m_volume_refactored = data_combined_15m_volume_refactored.fillna(1)
data_combined_60m_volume_refactored = data_combined_60m_volume_refactored.fillna(method='ffill')
data_combined_60m_volume_refactored = data_combined_60m_volume_refactored.fillna(method='bfill')
data_combined_60m_volume_refactored = data_combined_60m_volume_refactored.replace(0, 1)
data_combined_60m_volume_refactored = data_combined_60m_volume_refactored.fillna(1)
data_combined_daily_volume_refactored = data_combined_daily_volume_refactored.fillna(method='ffill')
data_combined_daily_volume_refactored = data_combined_daily_volume_refactored.fillna(method='bfill')
data_combined_daily_volume_refactored = data_combined_daily_volume_refactored.replace(0, 1)
data_combined_daily_volume_refactored = data_combined_daily_volume_refactored.fillna(1)

'''
data_combined_15m_OHLC_refactored = 0.25*(data_combined_15m_open_refactored + data_combined_15m_high_refactored + data_combined_15m_low_refactored + data_combined_15m_close_refactored)
data_combined_60m_OHLC_refactored = 0.25*(data_combined_60m_open_refactored + data_combined_60m_high_refactored + data_combined_60m_low_refactored + data_combined_60m_close_refactored)
data_combined_daily_OHLC_refactored = 0.25*(data_combined_daily_open_refactored + data_combined_daily_high_refactored + data_combined_daily_low_refactored + data_combined_daily_close_refactored)
'''


'\ndata_combined_15m_OHLC_refactored = 0.25*(data_combined_15m_open_refactored + data_combined_15m_high_refactored + data_combined_15m_low_refactored + data_combined_15m_close_refactored)\ndata_combined_60m_OHLC_refactored = 0.25*(data_combined_60m_open_refactored + data_combined_60m_high_refactored + data_combined_60m_low_refactored + data_combined_60m_close_refactored)\ndata_combined_daily_OHLC_refactored = 0.25*(data_combined_daily_open_refactored + data_combined_daily_high_refactored + data_combined_daily_low_refactored + data_combined_daily_close_refactored)\n'

Retain the last thousand rows.

In [58]:
window_15m = 7
window_60m = 7
window_daily = 7

nrows_15m = 1000 + window_15m
nrows_60m = 1000 + window_60m
nrows_daily = 1000 + window_daily

data_combined_15m_OHLC_refactored = data_combined_15m_OHLC_refactored.tail(nrows_15m)
data_combined_60m_OHLC_refactored = data_combined_60m_OHLC_refactored.tail(nrows_60m)
data_combined_daily_OHLC_refactored = data_combined_daily_OHLC_refactored.tail(nrows_daily)
'''
data_combined_15m_open_refactored = data_combined_15m_open_refactored.tail(nrows_15m)
data_combined_60m_open_refactored = data_combined_60m_open_refactored.tail(nrows_60m)
data_combined_daily_open_refactored = data_combined_daily_open_refactored.tail(nrows_daily)
data_combined_15m_high_refactored = data_combined_15m_high_refactored.tail(nrows_15m)
data_combined_60m_high_refactored = data_combined_60m_high_refactored.tail(nrows_60m)
data_combined_daily_high_refactored = data_combined_daily_high_refactored.tail(nrows_daily)
data_combined_15m_low_refactored = data_combined_15m_low_refactored.tail(nrows_15m)
data_combined_60m_low_refactored = data_combined_60m_low_refactored.tail(nrows_60m)
data_combined_daily_low_refactored = data_combined_daily_low_refactored.tail(nrows_daily)
data_combined_15m_close_refactored = data_combined_15m_close_refactored.tail(nrows_15m)
data_combined_60m_close_refactored = data_combined_60m_close_refactored.tail(nrows_60m)
data_combined_daily_close_refactored = data_combined_daily_close_refactored.tail(nrows_daily)
data_combined_15m_adjclose_refactored = data_combined_15m_adjclose_refactored.tail(nrows_15m)
data_combined_60m_adjclose_refactored = data_combined_60m_adjclose_refactored.tail(nrows_60m)
data_combined_daily_adjclose_refactored = data_combined_daily_adjclose_refactored.tail(nrows_daily)
'''
data_combined_15m_volume_refactored = data_combined_15m_volume_refactored.tail(nrows_15m)
data_combined_60m_volume_refactored = data_combined_60m_volume_refactored.tail(nrows_60m)
data_combined_daily_volume_refactored = data_combined_daily_volume_refactored.tail(nrows_daily)


Ignore this section. Just testing VWAP formula.

In [ ]:
ticker = symbols_dict['TICKER'][0]
window = 7
k0=20
a = data_combined_daily_close_refactored[ticker][k0-window:k0]
b = data_combined_daily_volume_refactored[ticker][k0-window:k0]
c = data_combined_daily_close_refactored[ticker][k0-window:k0] * data_combined_daily_volume_refactored[ticker][k0-window:k0]
print(a)
print(b)
print(c)
print(sum(c)/sum(b))

7-day VWAP for closing prices - all timeframes.

Note: currently slow as shit; will consider trying list comprehension instead; suggestions welcome.

In [59]:
t0 = datetime.datetime.now()

temp = pd.DataFrame().reindex_like(data_combined_15m_OHLC_refactored)
for ticker in temp.columns:
  #print(ticker,' 15m')
  for k0 in range(window_15m,temp.shape[0]):
        temp[ticker][k0] = sum(data_combined_15m_OHLC_refactored[ticker][k0-window_15m:k0] * data_combined_15m_volume_refactored[ticker][k0-window_15m:k0]) / sum(data_combined_15m_volume_refactored[ticker][k0-window_15m:k0])
data_combined_15m_OHLC_refactored_VWAP = temp

temp = pd.DataFrame().reindex_like(data_combined_60m_OHLC_refactored)
for ticker in temp.columns:
  #print(ticker,' 60m')
  for k0 in range(window_60m,temp.shape[0]):
        temp[ticker][k0] = sum(data_combined_60m_OHLC_refactored[ticker][k0-window_60m:k0] * data_combined_60m_volume_refactored[ticker][k0-window_60m:k0]) / sum(data_combined_60m_volume_refactored[ticker][k0-window_60m:k0])
data_combined_60m_OHLC_refactored_VWAP = temp

temp = pd.DataFrame().reindex_like(data_combined_daily_OHLC_refactored)
for ticker in temp.columns:
  #print(ticker,' daily')
  for k0 in range(window_daily,temp.shape[0]):
        temp[ticker][k0] = sum(data_combined_daily_OHLC_refactored[ticker][k0-window_daily:k0] * data_combined_daily_volume_refactored[ticker][k0-window_daily:k0]) / sum(data_combined_daily_volume_refactored[ticker][k0-window_daily:k0])
data_combined_daily_OHLC_refactored_VWAP = temp


data_combined_15m_OHLC_refactored_VWAP = data_combined_15m_OHLC_refactored_VWAP.dropna()
data_combined_60m_OHLC_refactored_VWAP = data_combined_60m_OHLC_refactored_VWAP.dropna()
data_combined_daily_OHLC_refactored_VWAP = data_combined_daily_OHLC_refactored_VWAP.dropna()


t0 = datetime.datetime.now() - t0
print('Section exec time: ', t0,'s')

Section exec time:  0:00:06.422710 s


In [105]:
data_combined_daily_OHLC_refactored_VWAP

LABEL,CRYPTO : XMR-USD,INDICES : ^OVX,INDICES : ^VIX,INDICES : ^VVIX,INDICES : ^VXN
idx,,,,,
2018-08-28,93.874609,24.377143,12.252500,95.540715,15.692857
2018-08-29,95.254989,24.208571,12.201429,94.451429,15.606786
2018-08-30,96.588308,24.011071,12.152500,93.519286,15.510357
2018-08-31,98.250977,23.990714,12.266786,93.206786,15.484643
2018-09-01,102.069477,24.102857,12.450000,93.248572,15.598929
...,...,...,...,...,...
2021-05-19,394.601780,38.381071,21.633571,125.337855,27.581786
2021-05-20,362.141994,39.044286,21.363214,124.555355,27.242500
2021-05-21,329.686541,38.998572,20.876428,124.538927,26.496786


Methods of correlation.

In [60]:
from scipy.stats import pearsonr

def calculate_pvalues(df):
    df = df.dropna()._get_numeric_data()
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    for r in df.columns:
        for c in df.columns:
            pvalues[r][c] = round(pearsonr(df[r], df[c])[1], 4)
    return pvalues

In [61]:
tick_interval = '60min'    # 15min, 60min, daily
lookback_period = data_combined_60m_OHLC_refactored_VWAP.shape[0]    # 30, 60, 90, 120, 180, df.shape[0]

if tick_interval == '15min':
  if lookback_period == 'all':
    lookback_period = data_combined_15m_OHLC_refactored_VWAP.shape[0]
  subset = data_combined_15m_OHLC_refactored_VWAP.tail(lookback_period)
if tick_interval == '60min':
  if lookback_period == 'all':
    lookback_period = data_combined_60m_OHLC_refactored_VWAP.shape[0]
  subset = data_combined_60m_OHLC_refactored_VWAP.tail(lookback_period)
if tick_interval == 'daily':
  if lookback_period == 'all':
    lookback_period = data_combined_daily_OHLC_refactored_VWAP.shape[0]
  subset = data_combined_daily_OHLC_refactored_VWAP.tail(lookback_period)

p_values = calculate_pvalues(subset) 
p2_values = p_values * 2    # alpha = 0.05
r_values = subset.corr()
r_values

LABEL,CRYPTO : XMR-USD,INDICES : ^OVX,INDICES : ^VIX,INDICES : ^VVIX,INDICES : ^VXN
LABEL,,,,,
CRYPTO : XMR-USD,1.000000,-0.280823,-0.172470,-0.327940,0.170076
INDICES : ^OVX,-0.280823,1.000000,-0.017335,-0.026315,-0.192019
INDICES : ^VIX,-0.172470,-0.017335,1.000000,0.970469,0.920864
INDICES : ^VVIX,-0.327940,-0.026315,0.970469,1.000000,0.821836
INDICES : ^VXN,0.170076,-0.192019,0.920864,0.821836,1.000000


In [62]:
correlations = r_values
for r in range(r_values.shape[0]):
  for c in range(r_values.shape[1]):
    if p2_values.iloc[r,c] > 0.05:
      correlations.iloc[r,c] = 0    # zero it if insignificant
correlations

LABEL,CRYPTO : XMR-USD,INDICES : ^OVX,INDICES : ^VIX,INDICES : ^VVIX,INDICES : ^VXN
LABEL,,,,,
CRYPTO : XMR-USD,1.000000,-0.280823,-0.172470,-0.327940,0.170076
INDICES : ^OVX,-0.280823,1.000000,0.000000,0.000000,-0.192019
INDICES : ^VIX,-0.172470,0.000000,1.000000,0.970469,0.920864
INDICES : ^VVIX,-0.327940,0.000000,0.970469,1.000000,0.821836
INDICES : ^VXN,0.170076,-0.192019,0.920864,0.821836,1.000000


In [63]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Heatmap(
        z=correlations,
        x=correlations.columns,
        y=correlations.columns,
        colorscale='Viridis'))

fig.update_layout(
    title='Cross Asset Correlations',
    xaxis_nticks=correlations.shape[0],
    yaxis_nticks=correlations.shape[0],
    width = 1000, height = 1000)

Hurst Exponent - Rolling

In [64]:
def hurst_it(series, window,return_type):

  error_flag = False
  H_array = np.array([])

  for k in range(window ,series.shape[0]):
    slice = series[k-window:k,]
    try:
      H, c, dataset = compute_Hc(slice, kind='price', simplified=True)
      H_array = np.append(H_array,H)
    except:
      H_array = np.append(H_array,np.NaN)
      error_flag = True
  
  if error_flag == False:
    H_array = np.append(H_array[0]*np.ones(window), H_array)
    # mean_line = np.mean(H_array)*np.ones(len(H_array))
  elif error_flag == True:
    H_array = np.append([np.NaN] * window, H_array)
    # mean_line = [None] * len(H_array)

  # mean_line = np.mean(H_array)*np.ones(len(H_array))
  mean_line = np.nanmean(H_array, axis=0) * np.ones(len(H_array))

  if return_type == 'hurst':
    return H_array
  elif return_type == 'mean':
    return mean_line

In [65]:
# test hurst
from scipy import signal
n = 1000
rando = 2.5 * np.random.random_sample((n,)) + 3
#sine = np.sin(list(range(1,1001)))
#t = np.linspace(0, 1, n, endpoint=False)
#sig = np.sin(2 * np.pi * t)
#sine = signal.square(2 * np.pi * 30 * t, duty=(sig + 1)/2)

now = datetime.datetime.now()
end = str(now.year)+"-"+str(now.month)+"-"+str(now.day)
start = now - datetime.timedelta(days=730-1)
NUH = yf.download('NUH.AX', start=start, end=end, interval='60m', prepost = True, threads = True, proxy = None)
NUH = NUH.Close.tail(1000)

bullish = np.array([0.5*sum(rando)])
bearish = np.array([0.5*sum(rando)])
for k in range(1,n):
  if np.random.rand()>0.4:
    bullish = np.append(bullish,bullish[-1]+rando[k])
    bearish = np.append(bearish,bearish[-1]-rando[k])
  else:
    bullish = np.append(bullish,bullish[-1]-rando[k])
    bearish = np.append(bearish,bearish[-1]+rando[k])

[*********************100%***********************]  1 of 1 completed


In [66]:
import plotly.graph_objects as go

posn = 2

fig = go.Figure()

fig.add_trace(go.Scatter(x=list(range(1,n+1)), y=bullish,
                    mode='lines',
                    name='bullish: ' + str(round(np.mean(hurst_it(bullish,len(bullish)-1,'hurst')),4))))

fig.update_layout(
    title='bullish: ' + str(round(np.mean(hurst_it(bullish,len(bullish)-1,'hurst')),4)),
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)

fig.show()

In [67]:
import plotly.graph_objects as go

posn = 2

fig = go.Figure()

fig.add_trace(go.Scatter(x=list(range(1,n+1)), y=bearish,
                    mode='lines',
                    name='bearish: ' + str(round(np.mean(hurst_it(bearish,len(bearish)-1,'hurst')),4))))

fig.update_layout(
    title='bearish: ' + str(round(np.mean(hurst_it(bearish,len(bearish)-1,'hurst')),4)),
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)

fig.show()

In [68]:
import plotly.graph_objects as go

posn = 2

fig = go.Figure()

fig.add_trace(go.Scatter(x=list(range(1,n+1)), y=NUH,
                    mode='lines',
                    name='NUH: ' + str(round(np.mean(hurst_it(NUH,len(NUH)-1,'hurst')),4))))

fig.update_layout(
    title='NUH: ' + str(round(np.mean(hurst_it(NUH,len(NUH)-1,'hurst')),4)),
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)

fig.show()

In [69]:
import plotly.graph_objects as go

posn = 2

fig = go.Figure()

fig.add_trace(go.Scatter(x=list(range(1,n+1)), y=rando,
                    mode='lines',
                    name='rando: ' + str(round(np.mean(hurst_it(rando,len(rando)-1,'hurst')),4))))

fig.update_layout(
    title='rando: ' + str(round(np.mean(hurst_it(rando,len(rando)-1,'hurst')),4)),
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)

fig.show()

In [70]:
hurst_combined_15m_OHLC = pd.DataFrame(index = data_combined_15m_OHLC_refactored_VWAP.index, columns = data_combined_15m_OHLC_refactored_VWAP.columns)
hurst_mean_combined_15m_OHLC = pd.DataFrame(index = data_combined_15m_OHLC_refactored_VWAP.index, columns = data_combined_15m_OHLC_refactored_VWAP.columns)

window = 128

for column in data_combined_15m_OHLC_refactored_VWAP.columns:
  series = data_combined_15m_OHLC_refactored_VWAP[column]
  temp_hurst = hurst_it(series, window, 'hurst')
  hurst_combined_15m_OHLC[column] = [x for x in temp_hurst]
  temp_hurst = hurst_it(series, window, 'mean')
  hurst_mean_combined_15m_OHLC[column] = [x for x in temp_hurst]


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning:

Mean of empty slice.



In [71]:
hurst_combined_60m_OHLC = pd.DataFrame(index = data_combined_60m_OHLC_refactored_VWAP.index, columns = data_combined_60m_OHLC_refactored_VWAP.columns)
hurst_mean_combined_60m_OHLC = pd.DataFrame(index = data_combined_60m_OHLC_refactored_VWAP.index, columns = data_combined_60m_OHLC_refactored_VWAP.columns)

window = 128

for column in data_combined_60m_OHLC_refactored_VWAP.columns:
  series = data_combined_60m_OHLC_refactored_VWAP[column]
  temp_hurst = hurst_it(series, window, 'hurst')
  hurst_combined_60m_OHLC[column] = [x for x in temp_hurst]
  temp_hurst = hurst_it(series, window, 'mean')
  hurst_mean_combined_60m_OHLC[column] = [x for x in temp_hurst]


In [72]:
hurst_combined_daily_OHLC = pd.DataFrame(index = data_combined_daily_OHLC_refactored_VWAP.index, columns = data_combined_daily_OHLC_refactored_VWAP.columns)
hurst_mean_combined_daily_OHLC = pd.DataFrame(index = data_combined_daily_OHLC_refactored_VWAP.index, columns = data_combined_daily_OHLC_refactored_VWAP.columns)

window = 128

for column in data_combined_daily_OHLC_refactored_VWAP.columns:
  series = data_combined_daily_OHLC_refactored_VWAP[column]
  temp_hurst = hurst_it(series, window, 'hurst')
  hurst_combined_daily_OHLC[column] = [x for x in temp_hurst]
  temp_hurst = hurst_it(series, window, 'mean')
  hurst_mean_combined_daily_OHLC[column] = [x for x in temp_hurst]


In [103]:
import plotly.graph_objects as go

posn = 2

fig = go.Figure()

fig.add_trace(go.Scatter(x=hurst_mean_combined_15m_OHLC.index, y=hurst_mean_combined_15m_OHLC.iloc[:,posn],
                    mode='lines',
                    name='mean Hurst'))
fig.add_trace(go.Scatter(x=hurst_combined_15m_OHLC.index, y=hurst_combined_15m_OHLC.iloc[:,posn],
                    mode='lines+markers',
                    name='100-period trailing Hurst: ' + hurst_combined_15m_OHLC.iloc[:,posn].name))

fig.show()

Fractal Dimension Index

In [77]:
FDI_combined_15m_OHLC = 2 - hurst_combined_15m_OHLC
FDI_combined_60m_OHLC = 2 - hurst_combined_60m_OHLC
FDI_combined_daily_OHLC = 2 - hurst_combined_daily_OHLC

In [144]:
import plotly.graph_objects as go

posn = 2

fig = go.Figure()

mean_line = np.mean(FDI_combined_15m_OHLC.iloc[:,posn])*np.ones(len(FDI_combined_15m_OHLC.iloc[:,posn]))

fig.add_trace(go.Scatter(x=FDI_combined_15m_OHLC.index, y=mean_line,
                    mode='lines',
                    name='mean FDI'))
fig.add_trace(go.Scatter(x=FDI_combined_15m_OHLC.index, y=FDI_combined_15m_OHLC.iloc[:,posn],
                    mode='lines+markers',
                    name='100-period trailing FDI: ' + FDI_combined_15m_OHLC.iloc[:,posn].name))
fig.show()

In [147]:
lookback = 30
alpha = 2 / (lookback +1)

ewm_combined_15m_OHLC = data_combined_15m_OHLC_refactored_VWAP.ewm(alpha = alpha, adjust = False).mean()
ewm_combined_60m_OHLC = data_combined_60m_OHLC_refactored_VWAP.ewm(alpha = alpha, adjust = False).mean()
ewm_combined_daily_OHLC = data_combined_daily_OHLC_refactored_VWAP.ewm(alpha = alpha, adjust = False).mean()

In [149]:
data_combined_15m_OHLC_refactored_VWAP

LABEL,CRYPTO : XMR-USD,INDICES : ^OVX,INDICES : ^VIX,INDICES : ^VVIX,INDICES : ^VXN
idx,,,,,
2021-05-16 02:45:00+01:00,387.937043,36.634999,20.2225,121.407501,26.3975
2021-05-16 03:00:00+01:00,389.614339,36.634999,20.2225,121.407501,26.3975
2021-05-16 03:15:00+01:00,389.019036,36.634999,20.2225,121.407501,26.3975
2021-05-16 03:30:00+01:00,389.114445,36.634999,20.2225,121.407501,26.3975
2021-05-16 03:45:00+01:00,390.184002,36.634999,20.2225,121.407501,26.3975
...,...,...,...,...,...
2021-05-22 23:45:00+01:00,235.577083,39.719999,20.1075,123.379999,24.2825
2021-05-23 00:00:00+01:00,235.577082,39.719999,20.1075,123.379999,24.2825
2021-05-23 00:15:00+01:00,235.717998,39.719999,20.1075,123.379999,24.2825


In [ ]:
ewm_combined_15m_OHLC.head(35)

In [145]:
import plotly.graph_objects as go

posn = 2

fig = go.Figure()

fig.add_trace(go.Scatter(x=ewm_combined_15m_OHLC.index, y=ewm_combined_15m_OHLC.iloc[:,posn],
                    mode='lines',
                    name=str(lookback) + '-period EWM'))
fig.add_trace(go.Scatter(x=data_combined_15m_OHLC_refactored_VWAP.index, y=data_combined_15m_OHLC_refactored_VWAP.iloc[:,posn],
                    mode='lines+markers',
                    name='OHLC VWAP'))
fig.show()

In [84]:
# http://www.stockspotter.com/Files/frama.pdf
# https://medium.com/geekculture/fractal-volatility-bands-new-trading-horizons-66ee445be198

def frama(price,FDI,window):

  frama_array = np.array([])
  frama_array = np.append(frama_array,price[0])

  for k in range(1,price.shape[0]):
    A = np.exp(-4.6 * (FDI[k] - 1))
    A = min(max(A,0.01),1)
    value = A * price[k] + (1 - A) * frama_array[k-1]
    frama_array = np.append(frama_array,value)

  return frama_array

In [85]:
frama_combined_15m_OHLC = pd.DataFrame(index = data_combined_15m_OHLC_refactored_VWAP.index, columns = data_combined_15m_OHLC_refactored_VWAP.columns)

for column in data_combined_15m_OHLC_refactored_VWAP.columns:
  price = ewm_combined_15m_OHLC[column]
  FDI = FDI_combined_15m_OHLC[column]
  temp_frama = frama(price,FDI,lookback)
  frama_combined_15m_OHLC[column] = [x for x in temp_frama]


In [86]:
frama_combined_60m_OHLC = pd.DataFrame(index = data_combined_60m_OHLC_refactored_VWAP.index, columns = data_combined_60m_OHLC_refactored_VWAP.columns)

for column in data_combined_60m_OHLC_refactored_VWAP.columns:
  price = ewm_combined_60m_OHLC[column]
  FDI = FDI_combined_60m_OHLC[column]
  temp_frama = frama(price,FDI,lookback)
  frama_combined_60m_OHLC[column] = [x for x in temp_frama]


In [87]:
frama_combined_daily_OHLC = pd.DataFrame(index = data_combined_daily_OHLC_refactored_VWAP.index, columns = data_combined_daily_OHLC_refactored_VWAP.columns)

for column in data_combined_daily_OHLC_refactored_VWAP.columns:
  price = ewm_combined_daily_OHLC[column]
  FDI = FDI_combined_daily_OHLC[column]
  temp_frama = frama(price,FDI,lookback)
  frama_combined_daily_OHLC[column] = [x for x in temp_frama]


In [110]:
frama_combined_15m_OHLC

LABEL,CRYPTO : XMR-USD,INDICES : ^OVX,INDICES : ^VIX,INDICES : ^VVIX,INDICES : ^VXN
idx,,,,,
2021-05-16 02:45:00+01:00,387.937043,36.634999,20.2225,121.407501,26.3975
2021-05-16 03:00:00+01:00,387.953680,NaN,NaN,NaN,NaN
2021-05-16 03:15:00+01:00,387.977419,NaN,NaN,NaN,NaN
2021-05-16 03:30:00+01:00,388.007490,NaN,NaN,NaN,NaN
2021-05-16 03:45:00+01:00,388.052886,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2021-05-22 23:45:00+01:00,236.554056,NaN,NaN,NaN,NaN
2021-05-23 00:00:00+01:00,236.452132,NaN,NaN,NaN,NaN
2021-05-23 00:15:00+01:00,236.368547,NaN,NaN,NaN,NaN


In [102]:
import plotly.graph_objects as go

posn = 2

fig = go.Figure()

fig.add_trace(go.Scatter(x=frama_combined_15m_OHLC.index, y=frama_combined_15m_OHLC.iloc[:,posn],
                    mode='lines',
                    name=str(lookback) + '-period FRAMA'))
fig.add_trace(go.Scatter(x=data_combined_15m_OHLC_refactored_VWAP.index, y=data_combined_15m_OHLC_refactored_VWAP.iloc[:,posn],
                    mode='lines+markers',
                    name='OHLC VWAP: ' + data_combined_15m_OHLC_refactored_VWAP.iloc[:,posn].name))
fig.show()

In [89]:
'''
def fractal_volatility_bands(series,window,std_multiple,return_type):

  vola = pd.DataFrame(std_multiple * volatility(series,window))
  vola.index = series.index

  if len(vola) == len(series):
    if return_type == 'lower':
      return series.subtract(vola[0], fill_value=0)
    elif return_type == 'upper':
      return series.add(vola[0], fill_value=0)
  else:
    return 0
'''


def volatility(series, window):

  std_array = np.array([])

  for k in range(window ,series.shape[0]):
    slice = series[k-window:k,]
    std = slice.std()
    std_array = np.append(std_array,std)
  
  std_array = np.append(std_array[0]*np.ones(window),std_array)
  return std_array


def bands(frama_series,price_series,window,std_multiple,return_type):

  a = pd.DataFrame(std_multiple*volatility(price_series,window))
  b = frama_series
  a.index = b.index

  if return_type == 'lower':
    return b.subtract(a[0], fill_value=0)
  elif return_type == 'upper':
    return b.add(a[0], fill_value=0)


In [90]:
lookback = 30

In [99]:
frama_combined_15m_OHLC

LABEL,CRYPTO : XMR-USD,INDICES : ^OVX,INDICES : ^VIX,INDICES : ^VVIX,INDICES : ^VXN
idx,,,,,
2021-05-16 02:45:00+01:00,387.937043,36.634999,20.2225,121.407501,26.3975
2021-05-16 03:00:00+01:00,387.953680,NaN,NaN,NaN,NaN
2021-05-16 03:15:00+01:00,387.977419,NaN,NaN,NaN,NaN
2021-05-16 03:30:00+01:00,388.007490,NaN,NaN,NaN,NaN
2021-05-16 03:45:00+01:00,388.052886,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2021-05-22 23:45:00+01:00,236.554056,NaN,NaN,NaN,NaN
2021-05-23 00:00:00+01:00,236.452132,NaN,NaN,NaN,NaN
2021-05-23 00:15:00+01:00,236.368547,NaN,NaN,NaN,NaN


In [91]:
posn = 2
frama_series = frama_combined_15m_OHLC.iloc[:,posn]
price_series = data_combined_15m_OHLC_refactored_VWAP.iloc[:,posn]
a = pd.DataFrame(2*volatility(price,lookback))
b = frama_series
a.index = b.index
c = b.subtract(a[0], fill_value=0)
d = b.add(a[0], fill_value=0)

ValueError: ignored

In [93]:
frama_combined_15m_lower = pd.DataFrame(index = data_combined_15m_OHLC_refactored_VWAP.index, columns = data_combined_15m_OHLC_refactored_VWAP.columns)
frama_combined_15m_upper = pd.DataFrame(index = data_combined_15m_OHLC_refactored_VWAP.index, columns = data_combined_15m_OHLC_refactored_VWAP.columns)

std_multiple = 2

for column in data_combined_15m_OHLC_refactored_VWAP.columns:
  frama_series = frama_combined_15m_OHLC[column]
  price_series = data_combined_15m_OHLC_refactored_VWAP[column]
  frama_combined_15m_lower[column] = bands(frama_series,price_series,lookback,std_multiple,'lower')
  frama_combined_15m_upper[column] = bands(frama_series,price_series,lookback,std_multiple,'upper')


In [94]:
frama_combined_60m_lower = pd.DataFrame(index = data_combined_60m_OHLC_refactored_VWAP.index, columns = data_combined_60m_OHLC_refactored_VWAP.columns)
frama_combined_60m_upper = pd.DataFrame(index = data_combined_60m_OHLC_refactored_VWAP.index, columns = data_combined_60m_OHLC_refactored_VWAP.columns)

std_multiple = 2

for column in data_combined_60m_OHLC_refactored_VWAP.columns:
  frama_series = frama_combined_60m_OHLC[column]
  price_series = data_combined_60m_OHLC_refactored_VWAP[column]
  frama_combined_60m_lower[column] = bands(frama_series,price_series,lookback,std_multiple,'lower')
  frama_combined_60m_upper[column] = bands(frama_series,price_series,lookback,std_multiple,'upper')


In [95]:
frama_combined_daily_lower = pd.DataFrame(index = data_combined_daily_OHLC_refactored_VWAP.index, columns = data_combined_daily_OHLC_refactored_VWAP.columns)
frama_combined_daily_upper = pd.DataFrame(index = data_combined_daily_OHLC_refactored_VWAP.index, columns = data_combined_daily_OHLC_refactored_VWAP.columns)

std_multiple = 2

for column in data_combined_daily_OHLC_refactored_VWAP.columns:
  frama_series = frama_combined_daily_OHLC[column]
  price_series = data_combined_daily_OHLC_refactored_VWAP[column]
  frama_combined_daily_lower[column] = bands(frama_series,price_series,lookback,std_multiple,'lower')
  frama_combined_daily_upper[column] = bands(frama_series,price_series,lookback,std_multiple,'upper')


In [100]:
import plotly.graph_objects as go

posn = 0

fig = go.Figure()

fig.add_trace(go.Scatter(x=frama_combined_15m_lower.index, y=frama_combined_15m_lower.iloc[:,posn],
                    mode='lines',
                    name=str(lookback) + '-period FRAMA lower band: ' + frama_combined_15m_lower.iloc[:,posn].name))
fig.add_trace(go.Scatter(x=frama_combined_15m_upper.index, y=frama_combined_15m_upper.iloc[:,posn],
                    mode='lines',
                    name=str(lookback) + '-period FRAMA upper band: ' + frama_combined_15m_upper.iloc[:,posn].name))
fig.add_trace(go.Scatter(x=frama_combined_15m_OHLC.index, y=frama_combined_15m_OHLC.iloc[:,posn],
                    mode='lines',
                    name=str(lookback) + '-period FRAMA: ' + frama_combined_15m_OHLC.iloc[:,posn].name))
fig.add_trace(go.Scatter(x=data_combined_15m_OHLC_refactored_VWAP.index, y=data_combined_15m_OHLC_refactored_VWAP.iloc[:,posn],
                    mode='lines+markers',
                    name='OHLC VWAP: ' + data_combined_15m_OHLC_refactored_VWAP.iloc[:,posn].name))
fig.show()

In [ ]:
import plotly.graph_objects as go

posn = 2

fig = go.Figure()

fig.add_trace(go.Scatter(x=frama_combined_60m_lower.index, y=frama_combined_60m_lower.iloc[:,posn],
                    mode='lines',
                    name=str(lookback) + '-period FRAMA lower band: ' + frama_combined_60m_lower.iloc[:,posn].name))
fig.add_trace(go.Scatter(x=frama_combined_60m_upper.index, y=frama_combined_60m_upper.iloc[:,posn],
                    mode='lines',
                    name=str(lookback) + '-period FRAMA upper band: ' + frama_combined_60m_upper.iloc[:,posn].name))
fig.add_trace(go.Scatter(x=frama_combined_60m_OHLC.index, y=frama_combined_60m_OHLC.iloc[:,posn],
                    mode='lines',
                    name=str(lookback) + '-period FRAMA: ' + frama_combined_60m_OHLC.iloc[:,posn].name))
fig.add_trace(go.Scatter(x=data_combined_60m_OHLC_refactored_VWAP.index, y=data_combined_60m_OHLC_refactored_VWAP.iloc[:,posn],
                    mode='lines+markers',
                    name='OHLC VWAP: ' + data_combined_60m_OHLC_refactored_VWAP.iloc[:,posn].name))
fig.show()

In [ ]:
import plotly.graph_objects as go

posn = 2

fig = go.Figure()

fig.add_trace(go.Scatter(x=frama_combined_daily_lower.index, y=frama_combined_daily_lower.iloc[:,posn],
                    mode='lines',
                    name=str(lookback) + '-period FRAMA lower band: ' + frama_combined_daily_lower.iloc[:,posn].name))
fig.add_trace(go.Scatter(x=frama_combined_daily_upper.index, y=frama_combined_daily_upper.iloc[:,posn],
                    mode='lines',
                    name=str(lookback) + '-period FRAMA upper band: ' + frama_combined_daily_upper.iloc[:,posn].name))
fig.add_trace(go.Scatter(x=frama_combined_daily_OHLC.index, y=frama_combined_daily_OHLC.iloc[:,posn],
                    mode='lines',
                    name=str(lookback) + '-period FRAMA: ' + frama_combined_daily_OHLC.iloc[:,posn].name))
fig.add_trace(go.Scatter(x=data_combined_daily_OHLC_refactored_VWAP.index, y=data_combined_daily_OHLC_refactored_VWAP.iloc[:,posn],
                    mode='lines+markers',
                    name='OHLC VWAP: ' + data_combined_daily_OHLC_refactored_VWAP.iloc[:,posn].name))
fig.show()

Facebook Prophet (using STAN)

In [ ]:
df = pd.DataFrame()
df['y'] = data_combined_daily_OHLC_refactored_VWAP['BTC-USD']
df['ds'] = df.index

m = Prophet()
m.fit(df)

look_forward_p = 0.05
periods = int(look_forward_p*df.shape[0])
future = m.make_future_dataframe(periods=periods)
forecast = m.predict(future)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [ ]:
import plotly.graph_objects as go

window_factor = 2
n = int(periods * window_factor)
thisSlice = forecast.tail(n)

fig = go.Figure()
fig.add_trace(go.Scatter(x=thisSlice.ds, y=thisSlice.yhat,
                    mode='lines',
                    name='yhat'))
fig.add_trace(go.Scatter(x=thisSlice.ds, y=thisSlice.yhat_lower,
                    mode='lines',
                    name='yhat_lower'))
fig.add_trace(go.Scatter(x=thisSlice.ds, y=thisSlice.yhat_upper,
                    mode='lines', 
                    name='yhat_upper'))

In [ ]:
periods

50

Ignore everything below...

In [ ]:
import pandas as pd

source_url = 'https://raw.githubusercontent.com/jefferygao1984/METIS_IDS_PROJECT/main/data_combined_daily.csv'
data = pd.read_csv(source_url)
data.head(5)

,idx,Date,Open,High,Low,Close,Adj_Close,Volume,TICKER
0,1998-12-22,1998-12-22,24.812500,25.06250,24.812500,25.03125,18.197704,5700,XLV
1,1998-12-23,1998-12-23,25.187500,25.59375,25.187500,25.59375,18.606638,18100,XLV
2,1998-12-24,1998-12-24,25.625000,25.75000,25.515625,25.75000,18.720226,4900,XLV
3,1998-12-28,1998-12-28,25.671875,25.75000,25.375000,25.37500,18.447601,15500,XLV
4,1998-12-29,1998-12-29,25.375000,25.93750,25.375000,25.93750,18.856541,5300,XLV


In [ ]:
data.TICKER.unique()

array(['XLV', 'XLP', 'XLU', 'XLK', 'XLY', 'XLB', 'XLI', 'XLE', 'XTL',
       'XOP', 'QQQ', 'HG=F', 'NG=F', 'SPHB', 'SPLV', 'MTUM', 'IWF', 'IWM',
       'MDY', 'CWB', 'HYG', 'BKLN', 'BIZD', 'FXE', 'JJC', 'PFF', 'CEW',
       'USO', 'VNQ', 'UNG', 'MUB', 'MBB', 'SHY', 'IEF', 'TLT', 'TIP',
       'LQD', 'UDN', 'SDY', 'DEF', 'QUAL', 'UUP', 'GDX', 'GDXJ', 'GLD',
       'VIXM', 'VIXY', 'BTC-USD', 'ETH-USD', 'XMR-USD', 'DOGE-USD'],
      dtype=object)

In [ ]:
subset = data[data['TICKER']=='BTC-USD']
subset_close = subset['Close']
subset_close.shape

(2420,)

In [ ]:
# !pip install hurst
from hurst import compute_Hc, random_walk

H, c, dataset = compute_Hc(subset_close, kind='price', simplified=True)
trend = c*dataset[0]**H

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=dataset[0], y=dataset[1],
                    mode='markers',
                    name='R/S series'))

fig.add_trace(go.Scatter(x=dataset[0], y=trend,
                    mode='lines',
                    name='trend'))

fig.update_xaxes(type="log")
fig.update_yaxes(type="log")

In [ ]:
# !pip install hurst
import pandas as pd
import numpy as np
from hurst import compute_Hc, random_walk

source_url = 'https://raw.githubusercontent.com/jefferygao1984/METIS_IDS_PROJECT/main/data_combined_daily.csv'
data = pd.read_csv(source_url)

subset = data[data['TICKER']=='XLE']
subset_close = subset['Close']

window = 180
H_array = np.array([])

for k in range(window,subset_close.shape[0]):
  slice = subset_close[k-window:k,]
  H, c, dataset = compute_Hc(slice, kind='price', simplified=True)
  H_array = np.append(H_array,H)

H_array = np.append(H_array[0]*np.ones(window),H_array)
subset['Hurst'] = H_array
mean_line = np.mean(H_array)*np.ones(len(H_array))

import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=subset['Date'], y=mean_line,
                    mode='lines',
                    name='mean Hurst'))
fig.add_trace(go.Scatter(x=subset['Date'], y=subset['Hurst'],
                    mode='lines+markers',
                    name='180-period trailing Hurst'))
fig.show()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
'''
import plotly.express as px

fig = px.line(subset, x="Date", y="Hurst", title= str(window) + '-period Hurst Exponent for ' + subset['TICKER'].iloc[0])
fig.show()
'''